## --- Day 8: Treetop Tree House ---
```
The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

30373
25512
65332
33549
35390

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

    The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
    The top-middle 5 is visible from the top and right.
    The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
    The left-middle 5 is visible, but only from the right.
    The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
    The right-middle 3 is visible from the right.
    In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?
```

In [1]:
from typing import List

In [2]:
with open("data/day_8.txt", "r") as f:
    grid = f.read().split("\n")

rows = {i: list(row) for i, row in enumerate(grid)}

In [3]:
def get_tree_from_grid(row: int, column: int) -> int:
    return int(rows[row][column])


assert get_tree_from_grid(0, 0) == 1
assert get_tree_from_grid(1, 1) == 2
assert get_tree_from_grid(5, 5) == 3

In [4]:
def get_trees_in_front(row: int, column: int):
    trees_in_front = {"above": [], "below": [], "left": [], "right": []}
    for row_ in range(row):
        trees_in_front["above"] = [get_tree_from_grid(row_, column)] + trees_in_front[
            "above"
        ]
    for column_ in range(column):
        trees_in_front["left"] = [get_tree_from_grid(row, column_)] + trees_in_front[
            "left"
        ]
    for row_ in range(row + 1, max(rows.keys()) + 1):
        trees_in_front["below"] = trees_in_front["below"] + [
            get_tree_from_grid(row_, column)
        ]
    for column_ in range(column + 1, len(rows[0])):
        trees_in_front["right"] = trees_in_front["right"] + [
            get_tree_from_grid(row, column_)
        ]
    return trees_in_front

In [5]:
def is_visible(tree_hight: int, trees_in_front: List[int]):
    return tree_hight > max(trees_in_front)


assert is_visible(5, [0, 1, 2, 3, 4])
assert not is_visible(5, [0, 1, 2, 3, 4, 5])
assert not is_visible(5, [0, 1, 2, 3, 4, 9])

In [6]:
visible_trees_in_interior = 0
for row in rows:
    if row > 0 and row < max(rows.keys()):
        for column, value in enumerate(rows[row]):
            if column > 0 and column < len(rows[0]) - 1:
                tree_height = get_tree_from_grid(row, column)
                trees_in_front = get_trees_in_front(row, column)
                visibility = []
                for direction in ["above", "below", "left", "right"]:
                    visibility.append(
                        is_visible(tree_height, trees_in_front[direction])
                    )
                visible_trees_in_interior += 1 if sum(visibility) > 0 else 0

In [7]:
visible_trees_from_perimeter = 2 * len(grid) + 2 * len(grid[0]) - 4

In [8]:
visible_trees_from_perimeter

392

In [9]:
visible_trees_in_interior

1298

In [10]:
total_trees_visible_from_outside = (
    visible_trees_from_perimeter + visible_trees_in_interior
)

In [11]:
total_trees_visible_from_outside

1690

In [12]:
assert total_trees_visible_from_outside == 1690

### --- Part Two ---
```
Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

30373
25512
65332
33549
35390

    Looking up, its view is not blocked; it can see 1 tree (of height 3).
    Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
    Looking right, its view is not blocked; it can see 2 trees.
    Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

30373
25512
65332
33549
35390

    Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
    Looking left, its view is not blocked; it can see 2 trees.
    Looking down, its view is also not blocked; it can see 1 tree.
    Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?
```

In [13]:
def how_many_trees_visible(tree_hight: int, trees_in_front: List[int]) -> int:
    n_trees = 1
    if not trees_in_front:
        return 0
    for tree in trees_in_front:
        if tree < tree_hight:
            n_trees += 1
        else:
            return n_trees
    return n_trees - 1


assert how_many_trees_visible(3, [1, 2, 3]) == 3
assert how_many_trees_visible(3, [3, 1, 1]) == 1
assert how_many_trees_visible(3, []) == 0
assert how_many_trees_visible(3, [1, 1]) == 2
assert how_many_trees_visible(2, [1, 1, 2, 4]) == 3

In [14]:
scenic_scores = []
for row in rows:
    if row > 0 and row < max(rows.keys()):
        for column, value in enumerate(rows[row]):
            if column > 0 and column < len(rows[0]) - 1:
                tree_height = get_tree_from_grid(row, column)
                trees_in_front = get_trees_in_front(row, column)
                score = 1
                for direction in ["above", "below", "left", "right"]:
                    score *= how_many_trees_visible(
                        tree_height, trees_in_front[direction]
                    )
                scenic_scores.append(score)

In [15]:
max(scenic_scores)

535680

In [16]:
assert max(scenic_scores) == 535680